In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
from matplotlib import pylab as plt
%matplotlib inline
import matplotlib.image as mpimg

In [2]:
from manage_tensorboard import init_directory, display_graph

In [3]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
# Initialise directory 
# for TensorBoard stuff
graph_directory = "tensorboard_graphs"
init_directory(graph_directory)

In [6]:

g1 = tf.Graph()
with g1.as_default():
    x = tf.placeholder(tf.float32, [None, 784],name="inputData")
    w = tf.Variable(tf.random_normal((784, 10), mean=0.0, stddev=0.01),name="weights")
    b = tf.Variable(tf.zeros(10),name="bias")
    y = tf.nn.softmax(tf.matmul(x, w) +b)#define a op node, ‘y’ 
    assert y.graph is g1
    y_ = tf.placeholder(tf.float32, [None, 10])
    with g1.name_scope("train"):
        with tf.control_dependencies([y]):
            cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
            train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
    with g1.name_scope("test"):
        with tf.control_dependencies([y]):
            correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    display_graph(g1.as_graph_def())    
                                   


In [7]:
with tf.Session( graph = g1 ):
    tf.initialize_all_variables().run()
    for i in range(1000):
        batch_xs, batch_ys = mnist.train.next_batch(100)
        train_step.run(feed_dict={x: batch_xs, y_: batch_ys})
    
    test_xs, test_ys = mnist.test.images,mnist.test.labels
    
    print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))


0.9163
